### Time Average

In [20]:
from numba import njit

@njit
def ac_time_average(x, max_tau):
    T = len(x)
    mean = np.mean(x)
    var = np.var(x)
    
    ac = np.zeros(max_tau+1)

    for tau in range(max_tau+1):
        prod = (x[:T-tau] - mean) * (x[tau:] - mean)
        ac[tau] = np.mean(prod) / var

    return ac

### Ensemble Average

In [11]:
def ac_ensemble_average(X, max_tau):
    N, T = X.shape
    ac = np.zeros(max_tau+1)
    d_ac = np.zeros(max_tau+1)

    means = np.mean(X, axis=1, keepdims=True)
    varsx = np.var(X, axis=1, keepdims=True)
    Xc = X - means

    for tau in range(max_tau+1):
        numeratori = np.sum(Xc[:,:T-tau]*Xc[:,tau:], axis=1)
        denominatori = (T-tau)*varsx.flatten()
        ac_tau = numeratori/denominatori
        ac[tau] = np.mean(ac_tau)
        d_ac[tau] = np.std(ac_tau)

    return ac, d_ac

### Mixed Average

In [18]:
def ac_mixed_average(X, max_tau):
    N, T = X.shape
    ac_N = np.zeros((N, max_tau+1))

    for i in range(N):
        mean = np.mean(X[i,:])
        var = np.var(X[i,:])
        
        for tau in range(max_tau+1):
            prod = (X[i, :T-tau] - mean) * (X[i, tau:] - mean)
            ac_N[i,tau]=np.mean(prod)/var

    ac = np.mean(ac_N, axis=0)
    d_ac = np.std(ac_N, axis=0)

    return ac, d_ac